# Redes Neurais Quânticas

## Introdução


As redes neurais artificiais são modelos computacionais para aprendizagem de máquina que ganharam significativa força nos últimos anos devido ao aumento do volume de dados e na capacidade de processamento de placas dedicadas. Esta tecnologia vem impactando todas as áreas de produção como agricultura, saúde, mineração, transportes, dentre outras. Dentre as possibilidade de plataformas para realização da computação das redes neurais artificiais, os computadores quânticos têm se mostrado uma possibilidade factível para gerar valor para essa área.

Uma propriedade da Mecânica Quântica é a de processar e armazenar grandes vetores e matrizes complexas e realizar operações lineares em tais vetores, resultando em um aumento exponencial na capacidade de desenvolvimento de redes neurais diretamente implementadas em um computador quântico.

O modelo mais simples de rede neural artificial foi proposto por Rosenblatt em 1957, uma vetor de valores reais I com dimensão m, que representa o input de informações, e um vetor  de valores reais W que representa os pesos da rede. O output da rede é dado pelo produto interno entre os vetores I e W que resulta numa probabilidade associada a uma decisão binária (sim/não). Nas implementações mais simples I e W possuem valores binários e apesar de serem limitados, são a base das redes neurais mais complexas que existem hoje em dia.

![](img/rede.png)


## Implementação do artigo "An artificial neuron implemented on an actual quantum processor"

No artigo “An artificial neuron implemented on an actual quantum processor” os autores propõem uma alternativa inspirada pela rede neural de Rosenblatt. Primeiramente os vetores de input e pesos de dimensão M são computados no computador quântico usando N qubits, de modo que M = 2^N. Isso explicita a vantagem informacional do computador quântico. Os autores também implementam um procedimento para gerar múltiplos estados de emaranhamento que permitiram diminuir os recursos computacionais necessários para gerar o algoritmo.
    De maneira prática, o sistema quântico é inicializado numa operação unitária Ui, que representa a entrada de dados, segue para um operação unitária Uw que representa os pesos da rede neural e o resultado é extraído por meio de um bit auxiliar (ancilla) que é usado para aplicar uma porta NOT multi controlada a fim de mensurar o estado de ativação do perceptron. A mensuração da ancila produz um output do estado ativado do perceptron com probabilidade |Cm−1|^2.
    
![](img/circuito.png)


Como observado no circuito acima, vamos definir o circuito Ui que será a entrada representativa dos dados. No exemplo demonstrado no artigo foram simuladas imagens 4x4 pixels que totalizam 16 pixels, cada um representando um valor binário (branco ou preto) que será implementado no algoritmo quântico por meio de uma inversão de sinal da porta lógica Z e CnZ.

[IMAGEM PORTA Z]

## Introdução ao Qiskit

........

In [1]:
import qiskit as qk

In [2]:
#=======================#
# INITIALIZATION
#======================#

# define nqubits (4+ancilla)
nqubits = 5

# creating a quantum register
q = qk.QuantumRegister(nqubits)

# creating a classical register 
c = qk.ClassicalRegister(nqubits)

# build quantum circuit with the qubits and classical register
circuit = qk.QuantumCircuit(q, c)

# print circuit
print(circuit)

         
q0_0: |0>
         
q0_1: |0>
         
q0_2: |0>
         
q0_3: |0>
         
q0_4: |0>
         
 c0_0: 0 
         
 c0_1: 0 
         
 c0_2: 0 
         
 c0_3: 0 
         
 c0_4: 0 
         


## Matriz Ui - Input de Dados da Rede Neural Quântica

In [3]:
#=======================#
# INPUT
#======================#

# Hadamard on all qubits but ancilla
for i in range(nqubits-1):
    circuit.h(q[i])
    
# Z-gate on first 3
for i in range(nqubits-2):
    circuit.z(q[i])
    
# Controlled Z
circuit.cz(q[1], q[2])
circuit.cz(q[0], q[2])
circuit.cz(q[0], q[1])
#circuit.ccz(q[0], q[1], q[2])

# print circuit
print(circuit)

         ┌───┐┌───┐         
q0_0: |0>┤ H ├┤ Z ├────■──■─
         ├───┤├───┤    │  │ 
q0_1: |0>┤ H ├┤ Z ├─■──┼──■─
         ├───┤├───┤ │  │    
q0_2: |0>┤ H ├┤ Z ├─■──■────
         ├───┤└───┘         
q0_3: |0>┤ H ├──────────────
         └───┘              
q0_4: |0>───────────────────
                            
 c0_0: 0 ═══════════════════
                            
 c0_1: 0 ═══════════════════
                            
 c0_2: 0 ═══════════════════
                            
 c0_3: 0 ═══════════════════
                            
 c0_4: 0 ═══════════════════
                            


# Uw

Como dito, a primeira fase do algoritmo consiste em realizar o produto interno entre um vetor i de entrada e outro vetor w de pesos. Para realizar isso, são usados dois operadores unitários. Um deles, Uw, tem a seguinte visualização geométrica



![](img/image01.png)

Primeiramente, em verde, é feita uma projeção do vetor |ψi⟩ no vetor |ψw⟩. Então, Uw rotaciona |ψw⟩ até alinhá-lo com |111...11⟩ e também rotaciona |ψi⟩ na mesma proporção. O produto interno i·w é representado pela projeção de Uw|ψi⟩ no eixo |111...11⟩ 

No artigo, são sugeridas duas abordagens: Força bruta a partir de sucessivas rotações e uma abordagem utilizando hipergrafos. A segunda, sugere uma definição pouco conhecida, mas nada mirabolante. Na verdade, a noção que temos de grafos nada mais são do que um caso especial dos hipergrafos.


![](img/grafohipergrafo.png)

Nos grafos, se as arestas são relacionamentos, esses relacionamentos são de 1:1. Ou seja, um relacionamento (uma aresta) não pode sair de um vértice em direção a vários outros. Nos hipergrafos, no entanto, isso é possível. Neles, uma aresta pode direcionar-se a vários outros vértices. Por isso, diz-se que os grafos são um caso especial de hipergrafos, em que arestas ligam somente dois vértices. 

![](img/hipergrafocircuito.png)

## Geração dos arquivos de Entrada

Conforme citado anteriormente, o neurônio proposto percebe com estímulo, ou recebe como entrada, um vetor $\vec{i}$, de tamanho $j$, formado pelos valores $i_j$ que corresponde aos pixels da imagem de entrada. Considerado uma imagem quadrada de tamanho 4, conforme a Figura abaixo, então é possível observar que cada pixel possui um valor binário que se refere a posição desse pixel. Adicionalmente, cada elemento da imagem possui um correspondente em $\vec{i}$, dessa forma, $i_0$ possuirá o valor da imagem na possição $0000$, enquanto que $i_1$ o valor em $0001$ e assim sucessivamente até $i_{j-1}$ que possuirá o valor da posição $1111$. É importante destacar que $i_j \in \{0,1\}$


![](img/exemplo.png)

Diante desse cenário, foram criadas duas funções para auxiliar na criação das imagens de entrada, a primeira chamada de $\textit{gerar_entrada_padrao(n)}$, essa função cria $({n-2})^2$ imagens, sendo $n$ o tamanho da imagem quadrada. Já a segunda função denominada, $\textit{gerar_entrada_aleatoria(n, m)}$, cria $n$ arquivos de saida, de forma que cada arquivo possui $m$ valores positivos, ou seja, com valor $1$.

In [12]:
import numpy as np
from random import randrange

#funcao para geracao dos arquivos de entrada com o padrao +
#input: 
#     n tamanho da imagem
#output:
#     arquivos com as matrizs geradas conforme o tamanho n definido
def gerar_entrada_padrao(n):
    for i in range (1, n-1):
        for j in range (1, n-1):
            imagem = np.zeros((n, n), dtype=np.uint8)
            imagem[i-1][j] = 1
            imagem[i][j] = 1
            imagem[i+1][j] = 1
            imagem[i][j] = 1
            imagem[i][j-1] = 1
            imagem[i][j] = 1
            imagem[i][j+1] = 1
            
            np.savetxt('input/input_{}_{}_{}.txt'.format(n, i, j), imagem, delimiter=' ', fmt='%s')

#funcao para geracao dos arquivos de entrada de forma aleatoria
#input: 
#     n tamanho da imagem
#     m quantidade de numeros aleatorios gerados
#output:
#     arquivos com as matrizs geradas conforme o tamanho n definido
def gerar_entrada_aleatoria(n, m):  
    for i in range (0, n):
        imagem = np.zeros((n, n), dtype=np.uint8)
        for j in range (0, m):
            r = randrange(n)
            s = randrange(n)
            imagem [r][s] = 1
    
        np.savetxt('input/input_{}.txt'.format(i), imagem, delimiter=' ', fmt='%s')

n = 4 #tamanho da imagem
m = 6 #quantidade de numeros aleatorios

gerar_entrada_padrao(n)
gerar_entrada_aleatoria(n, m)


Após a chamada das funções anteriores, é possível observar os arquivos gerados da seguinte forma:

In [27]:
arquivo_padrao = open('input/input_{}_1_1.txt'.format(n), 'r')

print(arquivo_padrao.read())

0 1 0 0
1 1 1 0
0 1 0 0
0 0 0 0

